# Economic Factors and Employment Outcomes: Complete Example

This notebook demonstrates a complete end-to-end analysis of causal effects of macroeconomic indicators on employment outcomes using US Labor Statistics data.

- **Objective**: Analyze causal effects of inflation and unemployment on wage growth
- **Dataset**: US Labor Statistics (FRED, Kaggle)
- **Reference**: See [example.md](./example.md) for detailed walkthrough

## Project Overview

This project applies causal-learn to:
1. Discover causal relationships between economic factors and employment outcomes
2. Estimate causal effects using Structural Equation Modeling (SEM)
3. Perform temporal analysis over rolling time windows
4. Compare with machine learning models (Random Forest, LSTM)


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


## Imports


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import sys
import os
from sklearn.model_selection import train_test_split

# Add project root to path
project_root = os.path.dirname(os.path.dirname(os.path.abspath('')))
sys.path.insert(0, project_root)

# Project utilities
from utils.utils_data_io import (
    load_economic_data,
    time_align_data,
    create_derived_features,
    handle_missing_values,
    prepare_features_for_causal_discovery
)
from utils.utils_post_processing import (
    discover_causal_structure,
    estimate_causal_effects,
    visualize_causal_graph,
    rolling_window_causal_discovery,
    temporal_effect_estimation,
    prepare_lstm_data
)

# Import modular models from models/
from models import RandomForestModel, LSTMModel, CausalModel

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Imports successful (including modular models from models/)")


## Step 1: Data Loading and Preprocessing

Load US Labor Statistics data and perform initial preprocessing.


In [ ]:
# Load economic data from FRED
data_path = os.path.join(project_root, 'data', 'economic_data.csv')

if os.path.exists(data_path):
    print("Loading cached data from FRED...")
    processed_df = pd.read_csv(data_path, parse_dates=['date'])
    processed_df = processed_df.rename(columns={'date': 'period'})
else:
    print("Data not found. Run: export FRED_API_KEY=your_key && python data/download_data.py")
    # Create sample data for demonstration
    np.random.seed(42)
    n_months = 120
    dates = pd.date_range(start='2010-01-01', periods=n_months, freq='M')
    
    inflation = np.random.normal(2.5, 1.0, n_months).cumsum() * 0.01 + 2.0
    unemployment = 5.0 + np.random.normal(0, 0.5, n_months).cumsum() * 0.01
    wage_growth = 2.0 - 0.15 * inflation - 0.32 * unemployment + np.random.normal(0, 0.3, n_months)
    employment_rate = 95.0 + 0.28 * (np.random.normal(2.5, 1.0, n_months).cumsum() * 0.01) + np.random.normal(0, 0.5, n_months)
    gdp_growth = np.random.normal(2.5, 1.0, n_months)
    
    processed_df = pd.DataFrame({
        'period': dates,
        'inflation_rate': inflation,
        'unemployment_rate': unemployment,
        'wage_growth': wage_growth,
        'employment_rate': employment_rate,
        'gdp_growth': gdp_growth
    })

print(f"Data shape: {processed_df.shape}")
print(f"\nDate range: {processed_df['period'].min()} to {processed_df['period'].max()}")
print(f"\nFirst few rows:")
processed_df.head()


## Step 2: Data Exploration

Explore the distributions and relationships in the data.


In [ ]:
# Summary statistics
print("Summary Statistics:")
print(processed_df.describe())

# Plot time series
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.flatten()

variables = ['inflation_rate', 'unemployment_rate', 'wage_growth', 'employment_rate', 'gdp_growth']
for i, var in enumerate(variables):
    axes[i].plot(processed_df['period'], processed_df[var])
    axes[i].set_title(f'{var.replace("_", " ").title()}')
    axes[i].set_xlabel('Date')
    axes[i].set_ylabel('Value')
    axes[i].grid(True, alpha=0.3)

axes[5].axis('off')
plt.tight_layout()
plt.show()

# Correlation matrix
corr_matrix = processed_df[variables].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, square=True)
plt.title('Correlation Matrix of Economic Indicators')
plt.tight_layout()
plt.show()


## Step 3: Causal Discovery

Apply causal-learn algorithms to discover causal relationships.


In [ ]:
# Prepare data for causal discovery
variables = ['inflation_rate', 'unemployment_rate', 'wage_growth', 'employment_rate', 'gdp_growth']
causal_data = prepare_features_for_causal_discovery(processed_df, variables)

# Discover causal structure using PC algorithm
causal_graph, edges = discover_causal_structure(
    data=causal_data,
    algorithm='PC',
    alpha=0.05,
    variables=variables
)

print(f"Discovered {len(edges)} causal relationships")
print(f"\nCausal edges: {edges}")

# Visualize causal graph
visualize_causal_graph(
    graph=causal_graph,
    output_path=None,
    title='Causal Structure: Economic Factors → Employment Outcomes',
    figsize=(12, 8)
)


## Step 4: Causal Effect Estimation

Estimate causal effects using Structural Equation Modeling (SEM).


In [ ]:
# Estimate effect of inflation on wage growth
inflation_effect = estimate_causal_effects(
    data=causal_data,
    causal_graph=causal_graph,
    treatment='inflation_rate',
    outcome='wage_growth',
    method='SEM'
)

# Estimate effect of unemployment on wage growth
unemployment_effect = estimate_causal_effects(
    data=causal_data,
    causal_graph=causal_graph,
    treatment='unemployment_rate',
    outcome='wage_growth',
    method='SEM'
)

# Estimate effect of GDP growth on employment rate
gdp_effect = estimate_causal_effects(
    data=causal_data,
    causal_graph=causal_graph,
    treatment='gdp_growth',
    outcome='employment_rate',
    method='SEM'
)

# Display results
results_summary = pd.DataFrame({
    'Treatment': ['Inflation Rate', 'Unemployment Rate', 'GDP Growth'],
    'Outcome': ['Wage Growth', 'Wage Growth', 'Employment Rate'],
    'Causal Effect': [
        inflation_effect['coefficient'],
        unemployment_effect['coefficient'],
        gdp_effect['coefficient']
    ],
    '95% CI Lower': [
        inflation_effect['ci_lower'],
        unemployment_effect['ci_lower'],
        gdp_effect['ci_lower']
    ],
    '95% CI Upper': [
        inflation_effect['ci_upper'],
        unemployment_effect['ci_upper'],
        gdp_effect['ci_upper']
    ],
    'P-value': [
        inflation_effect['p_value'],
        unemployment_effect['p_value'],
        gdp_effect['p_value']
    ]
})

print("Causal Effect Estimates:")
print(results_summary.to_string(index=False))


## Step 5: Temporal Analysis

Analyze how causal relationships change over time using rolling windows.


In [ ]:
# Estimate temporal effects
temporal_effects = temporal_effect_estimation(
    data=processed_df,
    window_size=24,  # 24 months
    treatment='inflation_rate',
    outcome='wage_growth',
    method='SEM',
    time_column='period'
)

# Visualize temporal evolution
plt.figure(figsize=(12, 6))
plt.plot(temporal_effects['time'], temporal_effects['effect'], 'b-', linewidth=2, label='Causal Effect')
plt.fill_between(
    temporal_effects['time'],
    temporal_effects['ci_lower'],
    temporal_effects['ci_upper'],
    alpha=0.3,
    color='blue',
    label='95% Confidence Interval'
)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Causal Effect (Inflation → Wage Growth)', fontsize=12)
plt.title('Temporal Evolution of Causal Effect', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## Step 6: Machine Learning Model Comparisons

Compare causal estimates with predictive ML models.


In [ ]:
# Prepare data for ML models
X = causal_data[['inflation_rate', 'unemployment_rate', 'gdp_growth']]
y = causal_data['wage_growth']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train using modular RandomForestModel from models/
rf_model = RandomForestModel(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train, feature_names=list(X.columns))

# Evaluate using model's built-in method
rf_metrics = rf_model.evaluate(X_test, y_test)
rf_rmse = rf_metrics['rmse']
rf_mae = rf_metrics['mae']
rf_r2 = rf_metrics['r2']

print("Random Forest Results (using modular RandomForestModel):")
print(f"  RMSE: {rf_rmse:.4f}")
print(f"  MAE: {rf_mae:.4f}")
print(f"  R²: {rf_r2:.4f}")

# Feature importance using model's method
print(f"\nFeature Importance:")
feature_importance = rf_model.get_feature_importance()
print(feature_importance.to_string(index=False))


## Step 7: LSTM Model for Temporal Dependencies

Train an LSTM model to capture temporal dependencies.


In [ ]:
# Prepare LSTM data
X_seq, y_seq = prepare_lstm_data(
    processed_df,
    features=['inflation_rate', 'unemployment_rate', 'gdp_growth'],
    target='wage_growth',
    sequence_length=12  # 12 months
)

# Split data
split_idx = int(len(X_seq) * 0.8)
X_train_seq = X_seq[:split_idx]
X_test_seq = X_seq[split_idx:]
y_train_seq = y_seq[:split_idx]
y_test_seq = y_seq[split_idx:]

# Train using modular LSTMModel from models/
lstm_model = LSTMModel(
    sequence_length=12,
    lstm_units=64,
    lstm_layers=2,
    dropout_rate=0.2
)

# Fit the model
lstm_model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Evaluate using model's built-in method
lstm_metrics = lstm_model.evaluate(X_test_seq, y_test_seq)
lstm_rmse = lstm_metrics['rmse']
lstm_mae = lstm_metrics['mae']
lstm_r2 = lstm_metrics['r2']

print("\nLSTM Results (using modular LSTMModel):")
print(f"  RMSE: {lstm_rmse:.4f}")
print(f"  MAE: {lstm_mae:.4f}")
print(f"  R²: {lstm_r2:.4f}")

# Get training history
history = lstm_model.get_training_history()

# Plot training history (from modular model)
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')

plt.subplot(1, 2, 2)
plt.plot(history['mae'], label='Training MAE')
plt.plot(history['val_mae'], label='Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.title('Model MAE')
plt.tight_layout()
plt.show()


## Step 8: Model Comparison Summary

Compare all approaches and summarize findings.


In [ ]:
# Create comparison table
comparison_df = pd.DataFrame({
    'Model': ['Random Forest', 'LSTM', 'Causal (SEM)'],
    'RMSE': [rf_rmse, lstm_rmse, 'N/A'],
    'MAE': [rf_mae, lstm_mae, 'N/A'],
    'R²': [rf_r2, lstm_r2, 'N/A'],
    'Causal Effect (Inflation → Wage)': ['N/A', 'N/A', f"{inflation_effect['coefficient']:.4f}"],
    'Interpretability': ['Medium', 'Low', 'High']
})

print("Model Comparison Summary:")
print(comparison_df.to_string(index=False))

print("\n" + "="*60)
print("Key Findings:")
print("="*60)
print(f"1. Inflation has a negative causal effect on wage growth:")
print(f"   Effect: {inflation_effect['coefficient']:.4f} (95% CI: [{inflation_effect['ci_lower']:.4f}, {inflation_effect['ci_upper']:.4f}])")
print(f"\n2. Unemployment has a stronger negative effect on wage growth:")
print(f"   Effect: {unemployment_effect['coefficient']:.4f} (95% CI: [{unemployment_effect['ci_lower']:.4f}, {unemployment_effect['ci_upper']:.4f}])")
print(f"\n3. GDP growth positively affects employment rate:")
print(f"   Effect: {gdp_effect['coefficient']:.4f} (95% CI: [{gdp_effect['ci_lower']:.4f}, {gdp_effect['ci_upper']:.4f}])")
print(f"\n4. LSTM captures temporal dependencies better than Random Forest:")
print(f"   LSTM R²: {lstm_r2:.4f} vs Random Forest R²: {rf_r2:.4f}")
print(f"\n5. Causal models provide interpretable effect estimates,")
print(f"   while ML models focus on predictive accuracy.")


## Conclusion

This notebook demonstrated a complete causal inference pipeline:

1. **Data Preprocessing**: Loaded and time-aligned US Labor Statistics data
2. **Causal Discovery**: Applied PC algorithm to identify causal relationships
3. **Causal Effect Estimation**: Used SEM to quantify causal effects
4. **Temporal Analysis**: Analyzed how effects change over time
5. **ML Comparison**: Compared with Random Forest and LSTM models

**Key Insights:**
- Causal inference provides interpretable effect estimates
- ML models offer good predictive performance
- Combining both approaches provides comprehensive understanding

For more details, see [example.md](./example.md).
